<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<br><b>Little Demo: Sky Backgrounds</b> <br>
Contact authors: Melissa Graham<br>
Last verified to run: 2023-09-22 <br>
LSST Science Pipelines version: Weekly 2023_37 <br>

Explore the sky backgrounds for a processed visit image (`calexp`) and a deeply coadded image (`deepCoadd`).

Import packages.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

Set plotting parameters.

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

Instantiate an instance of the butler.

In [ ]:
butler = Butler(config="dp02", collections="2.2i/runs/DP0.2")

<br>

**Sky background for a `calexp`.**

Obtain the `calexp` data for `detector` 75 in `visit` 512055.

In [ ]:
dataId = {'visit': 512055, 'detector': 75}
calexp = butler.get('calexp', **dataId)

Display the `calexp`. Note the maximum pixel value is about 400.

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

Retrieve the subtracted background.

In [ ]:
bkgd = butler.get('calexpBackground', **dataId)

Display the background image.

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.scale('linear', 'zscale')
afw_display.mtv(bkgd.getImage())

Get the array of pixel values and print statistics for the background.

Flatten the array and check for NaN values.

Use `numpy` to display basic array statistics.

In [ ]:
temp = np.asarray(bkgd.getImage().getArray(), dtype='float')
bkgd_array = temp.flatten()
print('Number of pixels:', len(bkgd_array))
del temp

tx = np.where( np.isnan(bkgd_array) )[0]
print('Number of NaN pixels: ', len(tx))
del tx

print('Minimum pixel value: ', np.min(bkgd_array))
print('Maximum pixel value: ', np.max(bkgd_array))
print('Mean pixel value: ', np.mean(bkgd_array))
print('Median pixel value: ', np.median(bkgd_array))
print('Standard deviation in pixel values: ', np.std(bkgd_array))

Show the histogram of pixel values.

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.hist(bkgd_array, bins=1000, log=True, color='grey',
         range=(np.nanmin(bkgd_array), np.nanmax(bkgd_array)))
plt.show()

Add the background back into the `calexp` and display it.

> **Warning:** Executing this cell more than once would add the background more than once.

In [ ]:
safety_tag = int(0)
if safety_tag == 0:
    calexp.maskedImage += bkgd.getImage()
    safety_tag += 1
else:
    print('WARNING: background has already been added. Did not re-add.')

Display the `calexp` with the background added.

Note that the maximum pixel count goes up to about 4000 (from 400 without the background).

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

Clean up.

In [ ]:
del dataId, calexp, bkgd, bkgd_array, safety_tag

<br>

**Sky background for a `deepCoadd_calexp` image.**

Sky backgrounds are subtracted from all processed visit images prior to coaddition.

A small final residual background is then subtracted to create the `deepCoadd_calexp` image.
The background of the `deepCoadd_calexp` image **does not** represent the 
combined backgrounds of the individual images that contributed to the coadded image.
It is these `deepCoadd_calexp` images which are used for source detection and measurement
and the creation of the multi-band `Object` catalog.

An image type called just `deepCoadd` is also created, and it does not have this
small final residual background subtracted.

Obtain the r-band `deepCoadd_calexp` and `deepCoadd` images for `tract` 4226 and `patch` 17.

In [ ]:
dataId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd_calexp = butler.get('deepCoadd_calexp', dataId)
coadd = butler.get('deepCoadd', dataId)

Option to list the visits which contributed to this coadded image.

In [ ]:
# coadd_calexp.getInfo().getCoaddInputs().visits.asAstropy()

Display the `deepCoadd_calexp`.

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(coadd_calexp.image)
plt.gca().axis('off')
plt.show()

Display the `deepCoadd`. No differences will be noticeable from the `deepCoadd_calexp`.

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(coadd.image)
plt.gca().axis('off')
plt.show()

Retrieve the small residual background subtracted to create the `deepCoadd_calexp` image.

In [ ]:
coadd_calexp_bkgd = butler.get('deepCoadd_calexp_background', dataId)

Display the background image. Notice how much flatter it appears than the `calexp_background`.

In [ ]:
fig = plt.figure(figsize=(4, 4))
afw_display = afwDisplay.Display(fig)
afw_display.mtv(coadd_calexp_bkgd.getImage())

Get the array of pixel values and print statistics for the background.
Flatten the array and check for NaN values.
Use `numpy` to display basic array statistics.

In [ ]:
temp = np.asarray(coadd_calexp_bkgd.getImage().getArray(), dtype='float')
bkgd_array = temp.flatten()
print('Number of pixels:', len(bkgd_array))
del temp

tx = np.where( np.isnan(bkgd_array) )[0]
print('Number of NaN pixels: ', len(tx))
del tx

print('Minimum pixel value: ', np.min(bkgd_array))
print('Maximum pixel value: ', np.max(bkgd_array))
print('Mean pixel value: ', np.mean(bkgd_array))
print('Median pixel value: ', np.median(bkgd_array))
print('Standard deviation in pixel values: ', np.std(bkgd_array))

Notice that the minimum, maximum, mean, and median are all the same value,
and that the standard deviation is zero. 
The small residual `deepCoadd_calexp` background is flat with no gradient.

Show that `deepCoadd` - `deepCoadd_calexp` $\approx$ `deepCoadd_calexp_background`.

In [ ]:
temp1 = np.asarray(coadd.getImage().getArray(), dtype='float')
temp2 = np.asarray(coadd_calexp.getImage().getArray(), dtype='float')
array = temp1.flatten() - temp2.flatten()

print('Minimum pixel value: ', np.min(array))
print('Maximum pixel value: ', np.max(array))
print('Mean pixel value: ', np.mean(array))
print('Median pixel value: ', np.median(array))
print('Standard deviation in pixel values: ', np.std(array))

These statistics are very similar to those for the `deepCoadd_calexp_background`.